In [1]:
# imports
from connection import create_spotify_oauth, get_token, get_tracks,get_audio_features, tracks_to_df,\
                        audio_features_to_df, get_artist_info, artist_info_to_df, get_album_info, album_info_to_df, extract_lists

import json

import pandas as pd
import numpy as np

In [2]:
# redirect uri, token and authentication variables
redirect_uri_string = 'localhost:5000/authorize/'

sp_ouath = create_spotify_oauth(redirect_uri_string)
token_info = get_token(redirect_uri_string)

In [3]:
# getting tracks
sources = ['5Gyrx7J2IbbQXbbKwa8bca', 
           '04j5JZf6upQSeYnWmcNKrH', 
           '5C9mrfuuQjE6KbceQfGhkj',
           '2YWkEfOW4TG0njIeLqFk4B',
           '2Rp5JvWfHIo3QqQPnmpEeD',
           '0fHyPDc90zfYysAWUklSaO',
           '4dQNbUHPWM2aia9upYNLew',
           '44SSFG0CDrXzafveLA4PCR',
           '02dSgnwgoovYRH6RlptgDC',
           '37i9dQZF1DWYiatDTQvsFP',
           '7jfRuO7rW49RJnafrxsJeu',
           '2UhVgdqKVqf9a365y1uh3V',
           '7kFayDb6PSfAavyuPZq1nw',
           '34uqcr7M0AKT1yfyhnZuUf',
           '5vpQYHxtoC0cy8GZMLQ35g']

all_tracks = []

for source in sources:
   track = get_tracks(source, token_info)
   all_tracks.extend(track['items'])

all_tracks_df = tracks_to_df(all_tracks)
all_tracks_df = all_tracks_df.drop_duplicates(subset=['track_id']).drop(columns=['artist_id', 'artist_names'])

In [4]:
# getting tracks' features
all_tracks_features = []

for track in all_tracks_df['track_id']:
    track_af = get_audio_features(track, token_info)
    all_tracks_features.extend(track_af)
    
all_tracks_features_df = audio_features_to_df(all_tracks_features)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=4B8gtpBuEtU9XUizl6BK2W:
 Max Retries, reason: too many 429 error responses

In [ ]:
# getting tracks' artists (WIP)
all_tracks_artists = []

for artist_id in all_tracks_df['artist_id']:
    for i in artist_id:
        artist_info = get_artist_info(i, token_info)
        all_tracks_artists.append(artist_info)

all_tracks_artists_df = artist_info_to_df(all_tracks_artists)

In [ ]:
# getting tracks' albums (WIP)
all_tracks_albums = []

for album_id in all_tracks_df['album_id']:
    album_info = get_album_info(album_id, token_info)
    all_tracks_albums.append(album_info)

# with open('all_tracks_albums.json', 'w') as outfile:
#     json.dump(all_tracks_albums, outfile, indent=4)

all_tracks_albums_df = album_info_to_df(all_tracks_albums)

In [ ]:
# preprocessing data
merged_tracks_df = pd.concat([all_tracks_df, all_tracks_features_df], axis=1) # merging tracks' info and features

merged_tracks_df = merged_tracks_df.loc[:, ~merged_tracks_df.columns.duplicated()] # dropping duplicate cols (track_id)

merged_tracks_df['album_date'] = pd.to_datetime(merged_tracks_df['album_date'], format='ISO8601') # changed album_date type to datetime

merged_tracks_df['explicit'] = merged_tracks_df['explicit'].replace({True: 1, 
                                                                     False: 0, 
                                                                     'nan': 0}) # encode explicit values, if nan assume not explicit



merged_tracks_df = merged_tracks_df.drop(columns=['track_name', 'album_name', 
                                                  'preview_url', 'album_cover_640', 
                                                  'album_cover_300','album_cover_64']) # dropping cols irrelevant to model

